In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [3]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


In [4]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.4.0
Python Version:     3.7.11
Operating System:   Darwin
Train Data Rows:    500
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting Au

Fitting model: NeuralNetTorch ...
	0.85	 = Validation score   (accuracy)
	1.56s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBMLarge ...
		`import lightgbm` failed. If you are using Mac OSX, Please try 'brew install libomp'. Detailed info: dlopen(/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)
Fitting model: WeightedEnsemble_L2 ...
	0.87	 = Validation score   (accuracy)
	0.13s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 30.05s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("agModels-predictClass/")


In [5]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [6]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8218855563517249
Evaluations on test data:
{
    "accuracy": 0.8218855563517249,
    "balanced_accuracy": 0.7209680177968016,
    "mcc": 0.47748810004400455,
    "f1": 0.5849236641221375,
    "precision": 0.6542155816435432,
    "recall": 0.5289042277825712
}


Predictions:  
 0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [7]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.842461,0.85,0.015351,0.005958,0.784596,0.015351,0.005958,0.784596,1,True,5
1,RandomForestGini,0.842461,0.84,0.116827,0.111533,0.388366,0.116827,0.111533,0.388366,1,True,3
2,RandomForestEntr,0.842358,0.84,0.114914,0.111875,0.391458,0.114914,0.111875,0.391458,1,True,4
3,ExtraTreesGini,0.834476,0.82,0.118612,0.112187,0.385980,0.118612,0.112187,0.385980,1,True,6
4,ExtraTreesEntr,0.831917,0.81,0.118125,0.112816,0.393539,0.118125,0.112816,0.393539,1,True,7
5,WeightedEnsemble_L2,0.821886,0.87,0.276344,0.127170,2.861255,0.002081,0.001261,0.126044,2,True,10
6,NeuralNetFastAI,0.818610,0.82,0.088348,0.010812,8.168602,0.088348,0.010812,8.168602,1,True,8
7,NeuralNetTorch,0.810523,0.85,0.140787,0.007135,1.557076,0.140787,0.007135,1.557076,1,True,9
8,KNeighborsUnif,0.725970,0.73,0.021705,0.002405,15.188296,0.021705,0.002405,15.188296,1,True,1
9,KNeighborsDist,0.695158,0.65,0.023597,0.002658,0.003321,0.023597,0.002658,0.003321,1,True,2


In [8]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

,<=50K,>50K
0,0.806976,0.193024
1,0.890178,0.109822
2,0.528649,0.471351
3,0.912868,0.087132
4,0.894868,0.105132


In [9]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2       0.87       0.127170   2.861255                0.001261           0.126044            2       True         10
1             CatBoost       0.85       0.005958   0.784596                0.005958           0.784596            1       True          5
2       NeuralNetTorch       0.85       0.007135   1.557076                0.007135           1.557076            1       True          9
3     RandomForestGini       0.84       0.111533   0.388366                0.111533           0.388366            1       True          3
4     RandomForestEntr       0.84       0.111875   0.391458                0.111875           0.391458            1       True          4
5      NeuralNetFastAI       0.82       0.010812   8.168602                0.010812           8.168602        

0:110: execution error: 找不到文件“某个对象”。 (-43)


In [10]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  binary
AutoGluon identified the following types of features:
('category', [])  : 7 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
('int', [])       : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
('int', ['bool']) : 1 | ['sex']


In [13]:
predictor.predict(test_data, model='CatBoost')

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object

In [14]:
time_limit = 60  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')
predictor.leaderboard(test_data, silent=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20220314_132531/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels/ag-20220314_132531/"
AutoGluon Version:  0.4.0
Python Version:     3.7.11
Operating System:   Darwin
Train Data Rows:    500
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_

Fitting model: NeuralNetTorch_BAG_L1 ... Training model for up to 51.2s of the 51.2s of remaining time.
	Fitting 5 child models (S1F1 - S1F5) | Fitting with SequentialLocalFoldFittingStrategy
	0.8489	 = Validation score   (roc_auc)
	6.2s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L1 ... Training model for up to 44.95s of the 44.95s of remaining time.
	Fitting 5 child models (S1F1 - S1F5) | Fitting with SequentialLocalFoldFittingStrategy
		`import lightgbm` failed. If you are using Mac OSX, Please try 'brew install libomp'. Detailed info: dlopen(/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)
Repeating

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.901983,0.898184,0.115257,0.067629,13.555085,0.115257,0.067629,13.555085,1,True,5
1,WeightedEnsemble_L2,0.899834,0.909102,3.835647,0.415602,41.609271,0.003048,0.000675,0.486114,2,True,10
2,RandomForestEntr_BAG_L1,0.886811,0.884739,0.115858,0.073408,0.384883,0.115858,0.073408,0.384883,1,True,4
3,RandomForestGini_BAG_L1,0.886119,0.878316,0.117030,0.070088,0.385798,0.117030,0.070088,0.385798,1,True,3
4,NeuralNetTorch_BAG_L1,0.883722,0.883917,2.324900,0.110817,21.275065,2.324900,0.110817,21.275065,1,True,9
5,ExtraTreesEntr_BAG_L1,0.881103,0.889488,0.115954,0.073341,0.384555,0.115954,0.073341,0.384555,1,True,7
6,NeuralNetFastAI_BAG_L1,0.880844,0.889599,1.275047,0.161802,5.907517,1.275047,0.161802,5.907517,1,True,8
7,ExtraTreesGini_BAG_L1,0.879546,0.894206,0.117395,0.074679,0.385489,0.117395,0.074679,0.385489,1,True,6
8,KNeighborsDist_BAG_L1,0.525998,0.536956,0.025608,0.001655,0.003204,0.025608,0.001655,0.003204,1,True,2
9,KNeighborsUnif_BAG_L1,0.514970,0.519604,0.022477,0.005961,0.026419,0.022477,0.005961,0.026419,1,True,1


In [15]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

Summary of age variable: 
 count    500.00000
mean      39.65200
std       13.52393
min       17.00000
25%       29.00000
50%       38.00000
75%       49.00000
max       85.00000
Name: age, dtype: float64


In [16]:
predictor_age = TabularPredictor(label=age_column, path="agModels-predictAge").fit(train_data, time_limit=60)
performance = predictor_age.evaluate(test_data)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "agModels-predictAge/"
AutoGluon Version:  0.4.0
Python Version:     3.7.11
Operating System:   Darwin
Train Data Rows:    500
Train Data Columns: 14
Label Column: age
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (85, 17, 39.652, 13.52393)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    735.57 MB
	Train Data (Original)  Memory Usage: 0.32 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify spec

Fitting model: NeuralNetTorch ... Training model for up to 57.74s of the 57.74s of remaining time.
	-11.9954	 = Validation score   (root_mean_squared_error)
	1.04s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBMLarge ... Training model for up to 56.69s of the 56.69s of remaining time.
		`import lightgbm` failed. If you are using Mac OSX, Please try 'brew install libomp'. Detailed info: dlopen(/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)
Fitting model: WeightedEnsemble_L2 ... Training model for up to 59.91s of the 56.4s of remaining time.
	-11.2431	 = Validation score   (root_mean_squared_error)
	0.1s	 = Training

In [17]:
predictor_age.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-10.572139,-11.243110,0.370990,0.125822,1.875892,0.002577,0.000208,0.101652,2,True,8
1,ExtraTreesMSE,-10.646028,-11.354060,0.118690,0.108943,0.278907,0.118690,0.108943,0.278907,1,True,5
2,RandomForestMSE,-10.755027,-11.660125,0.115939,0.112646,0.388159,0.115939,0.112646,0.388159,1,True,3
3,CatBoost,-10.780312,-11.799279,0.016711,0.004559,0.687825,0.016711,0.004559,0.687825,1,True,4
4,NeuralNetFastAI,-11.225699,-12.073282,0.085201,0.007624,0.447046,0.085201,0.007624,0.447046,1,True,6
5,NeuralNetTorch,-11.448391,-11.995427,0.139039,0.007127,1.044895,0.139039,0.007127,1.044895,1,True,7
6,KNeighborsUnif,-14.902058,-15.686937,0.021228,0.003065,0.006330,0.021228,0.003065,0.006330,1,True,1
7,KNeighborsDist,-15.771259,-15.180150,0.025483,0.001919,0.003392,0.025483,0.001919,0.003392,1,True,2


- 深度预测表中的列

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

import numpy as np

train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
print(train_data.head())

label = 'occupation'
print("Summary of occupation column: \n", train_data['occupation'].describe())

new_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
test_data = new_data[5000:].copy()  # this should be separate data in your applications
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])  # delete label column
val_data = new_data[:5000].copy()

metric = 'accuracy' # we specify eval-metric just for demo (unnecessary as it's the default)

       age workclass  fnlwgt      education  education-num  \
6118    51   Private   39264   Some-college             10   
23204   58   Private   51662           10th              6   
29590   40   Private  326310   Some-college             10   
18116   37   Private  222450        HS-grad              9   
33964   62   Private  109190      Bachelors             13   

            marital-status        occupation    relationship    race      sex  \
6118    Married-civ-spouse   Exec-managerial            Wife   White   Female   
23204   Married-civ-spouse     Other-service            Wife   White   Female   
29590   Married-civ-spouse      Craft-repair         Husband   White     Male   
18116        Never-married             Sales   Not-in-family   White     Male   
33964   Married-civ-spouse   Exec-managerial         Husband   White     Male   

       capital-gain  capital-loss  hours-per-week  native-country   class  
6118              0             0              40   United-State

In [3]:
import autogluon.core as ag

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data, tuning_data=val_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20220315_015030/"
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "AutogluonModels/ag-20220315_015030/"
AutoGluon Version:  0.4.0
Python Version:     3.7.11
Operating System:   Darwin
Train Data Rows:    500
Train Data Columns: 14
Tuning Data Rows:    5000
Tuning Data Columns: 14
Label Column: occupation
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  [' Exec-managerial', ' Other-service', ' Craft-repair', ' Sales', ' Prof-specialty', ' Protective-serv', ' ?', ' Adm-clerical', ' Machine-op-inspct', ' Tech-support']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 examples that w

  0%|          | 0/5 [00:00<?, ?it/s]

`import lightgbm` failed. If you are using Mac OSX, Please try 'brew install libomp'. Detailed info: dlopen(/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)
Traceback (most recent call last):
  File "/Users/yangzhou/.local/lib/python3.7/site-packages/autogluon/core/utils/try_import.py", line 93, in try_import_lightgbm
    import lightgbm
  File "/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/__init__.py", line 8, in <module>
    from .basic import Booster, Dataset, Sequence, register_logger
  File "/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/basic.py", line 110, in <module>
    _LIB = _load_lib()
  File "/Users/ya

`import lightgbm` failed. If you are using Mac OSX, Please try 'brew install libomp'. Detailed info: dlopen(/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)
Traceback (most recent call last):
  File "/Users/yangzhou/.local/lib/python3.7/site-packages/autogluon/core/utils/try_import.py", line 93, in try_import_lightgbm
    import lightgbm
  File "/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/__init__.py", line 8, in <module>
    from .basic import Booster, Dataset, Sequence, register_logger
  File "/Users/yangzhou/.local/lib/python3.7/site-packages/lightgbm/basic.py", line 110, in <module>
    _LIB = _load_lib()
  File "/Users/ya

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: NeuralNetTorch/T1 ...
	0.269	 = Validation score   (accuracy)
	4.49s	 = Training   runtime
	0.06s	 = Validation runtime
Fitted model: NeuralNetTorch/T2 ...
	0.316	 = Validation score   (accuracy)
	1.46s	 = Training   runtime
	0.09s	 = Validation runtime
Fitted model: NeuralNetTorch/T3 ...
	0.2983	 = Validation score   (accuracy)
	1.36s	 = Training   runtime
	0.14s	 = Validation runtime
Fitted model: NeuralNetTorch/T4 ...
	0.3371	 = Validation score   (accuracy)
	2.82s	 = Training   runtime
	0.19s	 = Validation runtime
Fitted model: NeuralNetTorch/T5 ...
	0.3258	 = Validation score   (accuracy)
	3.68s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 119.88s of the 103.95s of remaining time.
	0.351	 = Validation score   (accuracy)
	0.43s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 16.49s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, u

In [4]:
y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Predictions:   [' Exec-managerial', ' Exec-managerial', ' Craft-repair', ' Other-service', ' Adm-clerical']


Evaluation: accuracy on test data: 0.33424197945061856
Evaluations on test data:
{
    "accuracy": 0.33424197945061856
}


In [5]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.351035       0.577585  12.883433                0.001607           0.432759            2       True          6
1    NeuralNetTorch/T4   0.337092       0.185139   2.822023                0.185139           2.822023            1       True          4
2    NeuralNetTorch/T5   0.325815       0.233860   3.684330                0.233860           3.684330            1       True          5
3    NeuralNetTorch/T2   0.315973       0.094650   1.456319                0.094650           1.456319            1       True          2
4    NeuralNetTorch/T3   0.298339       0.137883   1.360654                0.137883           1.360654            1       True          3
5    NeuralNetTorch/T1   0.269018       0.062329   4.488002                0.062329           4.488002        

0:123: execution error: 找不到文件“某个对象”。 (-43)


### Multimodal Data Table:Tablular,text,and Image

In [ ]:
download_dir = './ag_petfinder_tutorial'
zip_file = 'https://automl-mm-bench.s3.amazonaws.com/petfinder_kaggle.zip'

In [ ]:
from autogluon.core.utils.loaders import load_zip
load_zip.unzip(zip_file,unzip_dir=download_dir)

In [ ]:
import os
os.listdir(download_dir)

In [ ]:
dataset_path = download_dir+'/petfinder_processed'
os.listdir(dataset_path)

In [ ]:
os.listdir(dataset_path+'/train_images')[:10]

In [ ]:
import pandas as pd
train_data = pd.read_csv(f'{dataset_path}/train.csv',index_col=0)
test_data = pd.read_csv(f'{dataset_path}/dev.csv',index_col=0)

In [ ]:
# 多分类标签
label = 'AdoptionSpeed'
image_col = 'Images'

In [ ]:
# 图像列
train_data[image_col].iloc[0]

In [ ]:
# 目前只支持每行一个图像
train_data[image_col] = train_data[image_col].apply(lambda ele:ele.split(';')[0])
test_data[image_col] = test_data[image_col].apply(lambda ele:ele.split(';')[0])
train_data[image_col].iloc[0]

In [ ]:
# 更新路径来指向磁盘上的正确位置
def path_expander(path,base_folder):
    path_l = path.split(';')
    return ';'.join([os.path.abspath(os.join(base_folder,path)) for path in path_l])
train_data[image_col] = train_data[image_col].apply(lambda ele:path_expander(ele,base_folder=dataset_path))
test_data[image_col] = test_data[image_col].apply(lambda ele:path_expander(ele,base_folder=dataset_path))
train_data[image_col].iloc[0]

In [ ]:
# 查看每行数据
example_row = train_data.iloc[1]
example_row

In [ ]:
# 文本信息
example_row['Description']

In [ ]:
# 图片信息
example_image = example_row['Images']
from IPython.display import Image,display
pil_img = Image(filename=example_image)
display(pil_img)

In [ ]:
# 选500条数据进行训练
train_data = train_data.sample(500,random_state=0)

In [ ]:
# 通过构造一个来自训练数据的FeatureMetadata对象来推断特性类型
from autogluon.tabular import FeatureMetadata
feature_metadata = FeatureMetadata.from_df(train_data)
print(feature_metadata)

In [ ]:
# 为了利用图像，告诉哪一列包含图像路径，通过FeatureMetadata对象并向图像列添加'image_path'特殊类型来实现
feature_metadata = feature_metadata.add_special_types({image_col:['image_path']})
print(feature_metadata)

In [ ]:
# 指定超参数
# AutoGluon有一个默认设置，可以很好地用于多模态数据集
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config
hyperparameters = get_hyperparameter_config('multimodal')
hyperparameters
# 这个超参数配置将训练各种列表模型，以及微调一个Electra BERT文本模型和一个ResNet图像模型

In [ ]:
# 在数据集上训练一个TabularPredicto，将同时利用表格、文本和图像特性
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=label).fit(train_data = train_data
                                             ,hyperparameters=hyperparameters
                                             ,feature_metadata=feature_metadata
                                             ,time_limit=900)

In [ ]:
# 在拟合之后，查看各种模型的表现
leaderboard = predictor.leaderboard(test_data)

#### 多模态数据表：结合BERT/Transformers和经典表格模型

In [1]:
# 处理包含文本/数字和分类列的多模式表格数据
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import random
from autogluon.tabular import TabularPredictor

In [ ]:
np.random.seed(123)
random.seed(123)

In [ ]:
# 产品情绪分析数据集
!mkdir -p product_sentiment_machine_hack

In [ ]:
subsample_size = 2000  # for quick demo, try setting to larger values
feature_columns = ['Product_Description', 'Product_Type']
label = 'Sentiment'

train_df = pd.read_csv('product_sentiment_machine_hack/train.csv', index_col=0).sample(2000, random_state=123)
dev_df = pd.read_csv('product_sentiment_machine_hack/dev.csv', index_col=0)
test_df = pd.read_csv('product_sentiment_machine_hack/test.csv', index_col=0)

train_df = train_df[feature_columns + [label]]
dev_df = dev_df[feature_columns + [label]]
test_df = test_df[feature_columns]
print('Number of training samples:', len(train_df))
print('Number of dev samples:', len(dev_df))
print('Number of test samples:', len(test_df))

In [ ]:
# 数据集有两个特征及标签
train_df.head()

In [ ]:
# 要使用TextPredictor内部的模型TabularPredictor，必须在AutoGluon表格中指定
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='Sentiment',path='ag_tabular_product_sentiment_multimodal')
predictor.fit(train_df,hyperparameters='multimodal')

In [ ]:
predictor.leaderboard(dev_df)

In [ ]:
# 使用Stack Ensemble 提高性能
predictor.fit(train_df,hyperparameters='multimodal',num_bag_folds=5,num_stack_levels=1)
# num_stack_levels(多少层叠层)和num_bag_folds(在袋装期间将数据分成多少折叠)
# 或者使用
predictor.fit(train_df,hyperparameters='multimodal',presets='best_quality')

#### 预测表中的多个列

In [1]:
from autogluon.tabular import TabularDataset,TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

In [ ]:
# 定义一个自定义MUltilabelPredictor类来管理一组TabularPredictor对象，每个标签一个
class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str, default = None
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str], default = None
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool, default = True
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [ ]:
# 训练
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

In [ ]:
labels = ['education-num','education','class']
problem_types = ['regression','multiclass','binary']
eval_metrics = ['mean_absolute_error','accuracy','accuracy']
save_path = 'agModels-predictEducationClass'
time_limit = 5

In [ ]:
multi_predictor = MultilabelPredictor(labels=labels,problem_types=problem_types,eval_metrics=eval_metrics,path=save_path)
multi_predictor.fit(train_data,time_limit=time_limit)

In [ ]:
# 预测和评估
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
test_datat = test_data.sample(n=subsample_size,random_state=0)
test_data_nolab = test_data.drop(columns=labels)
test_data_nolab.head()

In [ ]:
multi_predictor = Multilabelpredictor.load(save_path)
predictions = multi_predictor.predict(test_data_nolab)
print('Predictions: \n',predictions)

In [ ]:
# 如果新数据包含基本事实标签，还可以评估预测的性能
evaluations = multi_predictor.evaluate(test_data)
print(evaluations)
print('Evaluates using metrics:',multi_predictor.eval_metrics)

In [ ]:
# 访问一个标签的预测
predictor_class = multi_predictor.get_predictor('class')
predictor_class.leaderboard(silent=True)